In [1]:
import pathlib
import json

In [2]:
from textrec.paths import paths

In [3]:
import glob
import os
import json
import dateutil.parser
import datetime
import toolz

In [7]:
INVALID = (
    '3vf5fg', #(already done as 9fmfm4),
    '73qq5q', #(already done as 77j4mf),
    'ffhgxm', #(something messed up in analysis, logfile is out of sync)
)


In [11]:
def get_log_data(log_file, earliest):
    size = os.path.getsize(log_file)
    meta = None
    num_nexts = 0
    with open(log_file) as f:
        for idx, line in enumerate(f):
            if 'next' not in line and 'login' not in line and 'finalData' not in line:
                continue
            line = json.loads(line)
            if line.get('type') == 'next':
                num_nexts += 1
            elif line.get('type') == 'login':
                if 'jsTimestamp' in line:
                    timestamp = datetime.datetime.fromtimestamp(line['jsTimestamp'] / 1000)
                else:
                    timestamp = dateutil.parser.parse(line['timestamp'])
                if timestamp < earliest:
                    return
                platform_id = line['platform_id']
                meta = dict(
                    timestamp=timestamp,
                    config=line['config'],
                    platform_id=platform_id,
                    participant_id=line['participant_id'],
                    size=size,
                    complete=False) # will override
            elif line.get('type') == 'finalData':
#                 meta['finalData'] = line['finalData']
                meta['complete'] = True
    if meta:
        return dict(meta, num_nexts=num_nexts)


def get_logs(log_path, earliest):
    log_files = []
    for log_file in log_path.glob('*.jsonl'):
        data = get_log_data(log_file, earliest)
        if data is not None:
#             print(data)
            log_files.append(data)
    return log_files

In [12]:
log_files = get_logs(paths.top_level / 'logs-gcp1', earliest = datetime.datetime(2018, 5, 2))

In [13]:
not_invalid = [entry for entry in log_files if entry['participant_id'] not in INVALID]

In [14]:
did_some_work = [entry for entry in not_invalid if entry['num_nexts'] > 4] # arbitrary cutoff

In [15]:
incomplete = [entry for entry in did_some_work if not entry['complete']]

In [16]:
len(incomplete)

7

In [17]:
pd.DataFrame(incomplete)

,complete,config,num_nexts,participant_id,platform_id,size,timestamp
0,False,cap,9,3wjx7c,turkA1DL3NEZFY4V4E,287633,2018-05-11 12:31:27.764
1,False,cap,12,5xc2f3,turkA3QEVFM3UD67BA,240119,2018-05-02 12:37:34.886
2,False,cap,9,j2hq4v,turkA3QEVFM3UD67BA,78844,2018-05-02 13:16:35.184
3,False,cap,5,pmvwxg,turkA3GJWXA13GMS9H,8466,2018-05-16 20:15:05.341
4,False,cap,10,qwfvx5,turkA1DL3NEZFY4V4E,358967,2018-05-11 12:23:03.013
5,False,cap,5,rm9g6v,turkA326O3VWP8TO8N,5217,2018-05-16 15:29:23.444
6,False,cap,5,ww3222,None,15539,2018-05-11 06:27:59.416


In [22]:
complete = [entry for entry in not_invalid if entry['complete']]
complete.sort(key=lambda x: x['timestamp'])

In [23]:
' '.join(entry['participant_id'] for entry in complete)

'h52x67 jvccx2 36x2r3 gg65g6 692c8j qmwvwv 77j4mf 4ggxj8 5c39rx fvwhpc 26w4jv 7g8xw8 533r6c 74v545 vxjcf7 9f5xwx 3267ww wf4c3m 7jcm37 cf9p8m phqcw9 5jj59g gw3w72 559x69 gvwqp6'

In [24]:
# Dump a list of participant_ids
for config, group in toolz.groupby('config', complete).items():
    print()
    group = sorted(group, key=lambda x: x['timestamp'])
    print(f'{len(group)} completed in {config}')
    print(f'{config}:',  ' '.join(participant['participant_id'] for participant in group))


25 completed in cap
cap: h52x67 jvccx2 36x2r3 gg65g6 692c8j qmwvwv 77j4mf 4ggxj8 5c39rx fvwhpc 26w4jv 7g8xw8 533r6c 74v545 vxjcf7 9f5xwx 3267ww wf4c3m 7jcm37 cf9p8m phqcw9 5jj59g gw3w72 559x69 gvwqp6
